In [1]:
import glob
from pathlib import Path
from typing import List

import itertools

import pandas as pd
import numpy as np
import pglast

import logparsing
import action_generation.index_actions as index_actions

In [2]:
pgfiles = glob.glob(str(Path("artifacts/workload") / "postgresql*.csv"))
pgfiles = sorted(pgfiles)

## Get Tables and Columns

In [1]:
import db_connector
db_connector.get_existing_indexes()


[]

In [2]:
col_mappings = db_connector.get_col_mappings()

UndefinedTable: missing FROM-clause entry for table "class2"
LINE 3:         class.relname AS indname, class2.relname AS tablenam...
                                          ^

In [34]:
def get_idents(sql, after_where=True, ident_filter=None):
    if after_where and "WHERE" not in sql.upper():
        return []
    ind = sql.index("WHERE") if after_where else -1
    tokens = pglast.parser.scan(sql)
    if ident_filter is None:
        return [sql[tok.start:tok.end+1] for tok in tokens if tok.name == 'IDENT' and tok.start > ind]
    return [
        sql[tok.start:tok.end+1]
        for tok in tokens
        if tok.name == 'IDENT'
        and tok.start > ind
        and ident_filter(sql[tok.start:tok.end+1])
    ]

In [35]:
from collections import defaultdict
table_colrefs_joint_counts = defaultdict(lambda: defaultdict(np.uint64))
for pgfile in pgfiles:
  df = logparsing.parse_csv_log(pgfile)

  query_col_refs = df.queries.apply(get_idents, args=(True,lambda name : name in col_mappings))
  query_col_refs = query_col_refs[query_col_refs.str.len() != 0]
  tables_used = query_col_refs.apply(lambda l : set([col_mappings[c] for c in l]))
  for (cols, tables) in zip(query_col_refs, tables_used):
    for table in tables:
      cols_for_tabs = [col for col in cols if col_mappings[col] == table]
      joint_ref = tuple(dict.fromkeys(cols_for_tabs))
      table_colrefs_joint_counts[table][joint_ref] += 1

In [36]:
table_colrefs_joint_counts

defaultdict(<function __main__.<lambda>()>,
            {'customer': defaultdict(numpy.uint64,
                         {('C_W_ID', 'C_D_ID', 'C_ID'): 12062.0,
                          ('C_W_ID', 'C_D_ID', 'C_LAST', 'C_FIRST'): 2276.0}),
             'warehouse': defaultdict(numpy.uint64, {('W_ID',): 10522.0}),
             'district': defaultdict(numpy.uint64,
                         {('D_W_ID', 'D_ID'): 14435.0}),
             'item': defaultdict(numpy.uint64, {('I_ID',): 36175.0}),
             'stock': defaultdict(numpy.uint64,
                         {('S_I_ID', 'S_W_ID'): 72068.0,
                          ('S_W_ID', 'S_I_ID', 'S_QUANTITY'): 307.0}),
             'new_order': defaultdict(numpy.uint64,
                         {('NO_D_ID', 'NO_W_ID', 'NO_O_ID'): 3130.0,
                          ('NO_O_ID', 'NO_D_ID', 'NO_W_ID'): 3130.0}),
             'oorder': defaultdict(numpy.uint64,
                         {('O_ID', 'O_D_ID', 'O_W_ID'): 6260.0,
                          (

In [13]:
simple = index_actions.SimpleIndexGenerator(table_colrefs_joint_counts)
for action in simple:
  print(action)

CREATE INDEX idx_customer_CWID_CDID_CID ON customer  (C_W_ID, C_D_ID, C_ID);
CREATE INDEX idx_customer_CWID_CDID_CLAST_CFIRST ON customer  (C_W_ID, C_D_ID, C_LAST, C_FIRST);
CREATE INDEX idx_warehouse_WID ON warehouse  (W_ID);
CREATE INDEX idx_district_DWID_DID ON district  (D_W_ID, D_ID);
CREATE INDEX idx_item_IID ON item  (I_ID);
CREATE INDEX idx_stock_SIID_SWID ON stock  (S_I_ID, S_W_ID);
CREATE INDEX idx_stock_SWID_SIID_SQUANTITY ON stock  (S_W_ID, S_I_ID, S_QUANTITY);
CREATE INDEX idx_new_order_NODID_NOWID_NOOID ON new_order  (NO_D_ID, NO_W_ID, NO_O_ID);
CREATE INDEX idx_new_order_NOOID_NODID_NOWID ON new_order  (NO_O_ID, NO_D_ID, NO_W_ID);
CREATE INDEX idx_oorder_OID_ODID_OWID ON oorder  (O_ID, O_D_ID, O_W_ID);
CREATE INDEX idx_oorder_OWID_ODID_OCID_OID ON oorder  (O_W_ID, O_D_ID, O_C_ID, O_ID);
CREATE INDEX idx_order_line_OLOID_OLDID_OLWID ON order_line  (OL_O_ID, OL_D_ID, OL_W_ID);
CREATE INDEX idx_order_line_OLWID_OLDID_OLOID_OLIID ON order_line  (OL_W_ID, OL_D_ID, OL_O_ID, OL

In [14]:
exhaustive = index_actions.ExhaustiveIndexGenerator(table_colrefs_joint_counts, 2)
for action in exhaustive:
  print(action)

CREATE INDEX idx_customer_CWID ON customer  (C_W_ID);
CREATE INDEX idx_customer_CDID ON customer  (C_D_ID);
CREATE INDEX idx_customer_CID ON customer  (C_ID);
CREATE INDEX idx_customer_CLAST ON customer  (C_LAST);
CREATE INDEX idx_customer_CFIRST ON customer  (C_FIRST);
CREATE INDEX idx_customer_CWID_CDID ON customer  (C_W_ID, C_D_ID);
CREATE INDEX idx_customer_CWID_CID ON customer  (C_W_ID, C_ID);
CREATE INDEX idx_customer_CDID_CWID ON customer  (C_D_ID, C_W_ID);
CREATE INDEX idx_customer_CDID_CID ON customer  (C_D_ID, C_ID);
CREATE INDEX idx_customer_CID_CWID ON customer  (C_ID, C_W_ID);
CREATE INDEX idx_customer_CID_CDID ON customer  (C_ID, C_D_ID);
CREATE INDEX idx_customer_CWID_CLAST ON customer  (C_W_ID, C_LAST);
CREATE INDEX idx_customer_CWID_CFIRST ON customer  (C_W_ID, C_FIRST);
CREATE INDEX idx_customer_CDID_CLAST ON customer  (C_D_ID, C_LAST);
CREATE INDEX idx_customer_CDID_CFIRST ON customer  (C_D_ID, C_FIRST);
CREATE INDEX idx_customer_CLAST_CWID ON customer  (C_LAST, C_W_

In [15]:
typed = index_actions.TypedIndexGenerator(exhaustive)
for action in typed:
  print(action)

CREATE INDEX idx_customer_CWID ON customer USING HASH (C_W_ID);
CREATE INDEX idx_customer_CWID ON customer USING BRIN (C_W_ID);
CREATE INDEX idx_customer_CDID ON customer USING HASH (C_D_ID);
CREATE INDEX idx_customer_CDID ON customer USING BRIN (C_D_ID);
CREATE INDEX idx_customer_CID ON customer USING HASH (C_ID);
CREATE INDEX idx_customer_CID ON customer USING BRIN (C_ID);
CREATE INDEX idx_customer_CLAST ON customer USING HASH (C_LAST);
CREATE INDEX idx_customer_CLAST ON customer USING BRIN (C_LAST);
CREATE INDEX idx_customer_CFIRST ON customer USING HASH (C_FIRST);
CREATE INDEX idx_customer_CFIRST ON customer USING BRIN (C_FIRST);
CREATE INDEX idx_customer_CWID_CDID ON customer USING HASH (C_W_ID, C_D_ID);
CREATE INDEX idx_customer_CWID_CDID ON customer USING BRIN (C_W_ID, C_D_ID);
CREATE INDEX idx_customer_CWID_CID ON customer USING HASH (C_W_ID, C_ID);
CREATE INDEX idx_customer_CWID_CID ON customer USING BRIN (C_W_ID, C_ID);
CREATE INDEX idx_customer_CDID_CWID ON customer USING HA